In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.chains import LLMChain
import openai
import graphviz


In [2]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Retrieve OpenAI API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Validate the key
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API Key not found! Check your .env file.")

print(f"Loaded OpenAI API Key: {OPENAI_API_KEY[:5]}********")  # Masked for security


Loaded OpenAI API Key: sk-pr********


In [3]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

C:\Users\admin\AppData\Local\Temp\ipykernel_11756\535739686.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)


In [4]:
import openai

# Set API key
openai.api_key = OPENAI_API_KEY

# Test with OpenAI
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

print(response.choices[0].message.content)


Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name="gpt-4", temperature=0.7)

few_shot_examples = """
Example 1:
Problem: A user wants to book a flight online.
Goals: Book a flight, receive confirmation.
Primary Actor: User.
Secondary Actors: Airline System, Payment Gateway.
Interactions: 
1. User selects flight.
2. User enters details.
3. Payment gateway processes payment.
4. Airline system confirms booking.

Example 2:
Problem: An online food delivery system.
Goals: Order food, track delivery.
Primary Actor: Customer.
Secondary Actors: Restaurant, Delivery Agent, Payment Gateway.
Interactions:
1. Customer selects food.
2. Customer makes payment.
3. Restaurant prepares food.
4. Delivery agent delivers food.
"""

prompt_template = PromptTemplate(
    input_variables=["problem_statement"],
    template="""
Analyze the problem statement and extract the following:
- Goals
- Primary and Secondary Actors
- Interactions between actors

Use the provided examples as reference.

Problem Statement: {problem_statement}
---
Few-shot Examples:
{few_shot_examples}
"""
)

chain = LLMChain(llm=llm, prompt=prompt_template)

def extract_info(problem_statement):
    response = chain.run({"problem_statement": problem_statement, "few_shot_examples": few_shot_examples})
    return response


C:\Users\admin\AppData\Local\Temp\ipykernel_11756\3651862265.py:48: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


In [6]:
import graphviz

def generate_use_case_diagram(actors, interactions):
    dot = graphviz.Digraph()

    # Add primary and secondary actors
    for actor in actors["primary"]:
        dot.node(actor, shape="ellipse", color="red")

    for actor in actors["secondary"]:
        dot.node(actor, shape="ellipse", color="blue")

    # Add interactions
    for interaction in interactions:
        dot.edge(interaction["actor"], interaction["use_case"], label=interaction["description"])

    return dot


In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template = PromptTemplate(
    input_variables=["use_case"],
    template="Describe the use case '{use_case}' in detail."
)

chain = LLMChain(llm=llm, prompt=prompt_template)

def generate_use_case_descriptions(use_cases):
    return {use_case: chain.run({"use_case": use_case}) for use_case in use_cases}


In [8]:
def process_problem_statement(problem_statement):
    # Extract goals, actors, and interactions
    response = extract_info(problem_statement)
    print("Extracted Data:\n", response)

    # Convert response to structured format (Modify as needed)
    actors = {
        "primary": ["User"],
        "secondary": ["Doctor", "Appointment System", "Payment Gateway"]
    }
    interactions = [
        {"actor": "User", "use_case": "Book Appointment", "description": "User selects doctor and time slot"},
        {"actor": "Appointment System", "use_case": "Confirm Appointment", "description": "System confirms booking"},
        {"actor": "Payment Gateway", "use_case": "Process Payment", "description": "User completes payment"},
        {"actor": "Doctor", "use_case": "Provide Consultation", "description": "Doctor consults patient"}
    ]

    # Generate and display use case diagram
    diagram = generate_use_case_diagram(actors, interactions)
    diagram.render("use_case_diagram", format="png", cleanup=False)
    
    # Generate descriptions
    use_case_descriptions = generate_use_case_descriptions(
        ["Book Appointment", "Confirm Appointment", "Process Payment", "Provide Consultation"]
    )

    return {
        "actors": actors,
        "interactions": interactions,
        "diagram": diagram,
        "descriptions": use_case_descriptions
    }

if __name__ == "__main__":
    problem_statement = "A user wants to schedule an online doctor appointment."
    result = process_problem_statement(problem_statement)


C:\Users\admin\AppData\Local\Temp\ipykernel_11756\3651862265.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"problem_statement": problem_statement, "few_shot_examples": few_shot_examples})


ValueError: Missing some input keys: {'use_case'}